## Hugging face kpi model

This notebook first tries zero short learning with a bert model or in other words, direct prediction with a bert model on the climate relevance task. Then it fine tunes the bert model for the relevance task using the huggingface transformers package. 

In [70]:
import config
import os
import pathlib
from dotenv import load_dotenv
from src.data.s3_communication import S3Communication
import numpy as np
from transformers import DistilBertTokenizerFast
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForQuestionAnswering
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer
import json
from torch import cuda
import collections
from tqdm.auto import tqdm
import torch
from transformers import default_data_collator
device = 'cuda' if cuda.is_available() else 'cpu'

In [2]:
# Load credentials
dotenv_dir = os.environ.get(
    "CREDENTIAL_DOTENV_DIR", os.environ.get("PWD", "/opt/app-root/src")
)
dotenv_path = pathlib.Path(dotenv_dir) / "credentials.env"
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path, override=True)

In [3]:
# init s3 connector
s3c = S3Communication(
    s3_endpoint_url=os.getenv("S3_ENDPOINT"),
    aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
    s3_bucket=os.getenv("S3_BUCKET"),
)

## Retrieve the test dataset and the trained models

In [4]:
s3c.download_files_in_prefix_to_dir(
    config.BASE_TRAIN_TEST_DATASET_S3_PREFIX,
    config.BASE_PROCESSED_DATA)

In [5]:
test_data_path = str(config.BASE_PROCESSED_DATA)+'/kpi_test_split.json'
train_data_path = str(config.BASE_PROCESSED_DATA)+'/kpi_train_split.json'

train_processed_data_path = str(config.BASE_PROCESSED_DATA)+'/kpi_processed_train_split.json'
test_processed_data_path = str(config.BASE_PROCESSED_DATA)+'/kpi_processed_test_split.json'

In [6]:
def get_process_raw_data(data_path, output_path):
    with open(data_path, "r") as read_file:
        data = json.load(read_file)
    squad = list()
    for instance in data['data']:
        instance_dic = dict()
        title = instance['title']
        for para in instance['paragraphs']:
            context = para['context']
            for ques in para['qas']:
                question = ques['question']
                qid = ques['id']
                #is_impossible = ques['is_impossible']
                answers = ques['answers']
                instance_dic = {'id': qid,
                                'title': title,
                                'context': context,
                                'question': question,
                                'answers': answers}
                squad.append(instance_dic)
    with open(output_path, "w") as write_file:
        json.dump(squad, write_file)


get_process_raw_data(train_data_path, train_processed_data_path)
get_process_raw_data(test_data_path, test_processed_data_path)

In [12]:
climate_dataset = load_dataset("json", data_files=train_processed_data_path)
climate_dataset['test'] = load_dataset("json", data_files=test_processed_data_path)['train']

Using custom data configuration default-343a66958535ceab
Found cached dataset json (/opt/app-root/src/.cache/huggingface/datasets/json/default-343a66958535ceab/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-f5901488e869dc33
Found cached dataset json (/opt/app-root/src/.cache/huggingface/datasets/json/default-f5901488e869dc33/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)


  0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
model_checkpoint = "distilbert-base-uncased"
batch_size = 16
tokenizer = DistilBertTokenizerFast.from_pretrained(model_checkpoint)
pad_on_right = tokenizer.padding_side == "right"
max_length = 384  #  The maximum length of a feature (question and context)
doc_stride = 128  #  The authorized overlap between two part of the context when splitting it is needed.

In [9]:
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []
    print(type(tokenized_examples))

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers[0]["answer_start"]
            end_char = start_char + len(answers[0]["text"])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [13]:
climate_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 66220
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 16891
    })
})

In [14]:
tokenized_datasets = climate_dataset.map(prepare_train_features,
                                         batched=True,
                                         remove_columns=climate_dataset["train"].column_names)

  0%|          | 0/67 [00:00<?, ?ba/s]

<class 'transformers.tokenization_utils_base.BatchEncoding'>
<class 'transformers.tokenization_utils_base.BatchEncoding'>
<class 'transformers.tokenization_utils_base.BatchEncoding'>
<class 'transformers.tokenization_utils_base.BatchEncoding'>
<class 'transformers.tokenization_utils_base.BatchEncoding'>
<class 'transformers.tokenization_utils_base.BatchEncoding'>
<class 'transformers.tokenization_utils_base.BatchEncoding'>
<class 'transformers.tokenization_utils_base.BatchEncoding'>
<class 'transformers.tokenization_utils_base.BatchEncoding'>
<class 'transformers.tokenization_utils_base.BatchEncoding'>
<class 'transformers.tokenization_utils_base.BatchEncoding'>
<class 'transformers.tokenization_utils_base.BatchEncoding'>
<class 'transformers.tokenization_utils_base.BatchEncoding'>
<class 'transformers.tokenization_utils_base.BatchEncoding'>
<class 'transformers.tokenization_utils_base.BatchEncoding'>
<class 'transformers.tokenization_utils_base.BatchEncoding'>
<class 'transformers.tok

  0%|          | 0/17 [00:00<?, ?ba/s]

<class 'transformers.tokenization_utils_base.BatchEncoding'>
<class 'transformers.tokenization_utils_base.BatchEncoding'>
<class 'transformers.tokenization_utils_base.BatchEncoding'>
<class 'transformers.tokenization_utils_base.BatchEncoding'>
<class 'transformers.tokenization_utils_base.BatchEncoding'>
<class 'transformers.tokenization_utils_base.BatchEncoding'>
<class 'transformers.tokenization_utils_base.BatchEncoding'>
<class 'transformers.tokenization_utils_base.BatchEncoding'>
<class 'transformers.tokenization_utils_base.BatchEncoding'>
<class 'transformers.tokenization_utils_base.BatchEncoding'>
<class 'transformers.tokenization_utils_base.BatchEncoding'>
<class 'transformers.tokenization_utils_base.BatchEncoding'>
<class 'transformers.tokenization_utils_base.BatchEncoding'>
<class 'transformers.tokenization_utils_base.BatchEncoding'>
<class 'transformers.tokenization_utils_base.BatchEncoding'>
<class 'transformers.tokenization_utils_base.BatchEncoding'>
<class 'transformers.tok

## Train

In [17]:
interim_model_checkpoint = "distilbert-base-uncased-finetuned-squad/checkpoint-4500"
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint).to(device)

model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-squad",
    evaluation_strategy = "no",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    weight_decay=0.01,
    push_to_hub=False,
)

data_collator = default_data_collator

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=None,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_transform.weight', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this mode

In [61]:
trainer.train()

/opt/app-root/lib64/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 66449
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 4154
  Number of trainable parameters = 66364418


Step,Training Loss
500,0.324900
1000,0.077500
1500,0.056400
2000,0.045500
2500,0.052600
3000,0.047300
3500,0.045700
4000,0.037500


Saving model checkpoint to distilbert-base-uncased-finetuned-squad/checkpoint-500
Configuration saved in distilbert-base-uncased-finetuned-squad/checkpoint-500/config.json
Model weights saved in distilbert-base-uncased-finetuned-squad/checkpoint-500/pytorch_model.bin
tokenizer config file saved in distilbert-base-uncased-finetuned-squad/checkpoint-500/tokenizer_config.json
Special tokens file saved in distilbert-base-uncased-finetuned-squad/checkpoint-500/special_tokens_map.json
Saving model checkpoint to distilbert-base-uncased-finetuned-squad/checkpoint-1000
Configuration saved in distilbert-base-uncased-finetuned-squad/checkpoint-1000/config.json
Model weights saved in distilbert-base-uncased-finetuned-squad/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in distilbert-base-uncased-finetuned-squad/checkpoint-1000/tokenizer_config.json
Special tokens file saved in distilbert-base-uncased-finetuned-squad/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to 

TrainOutput(global_step=4154, training_loss=0.08397273301045138, metrics={'train_runtime': 821.3564, 'train_samples_per_second': 80.902, 'train_steps_per_second': 5.057, 'total_flos': 6511325883019776.0, 'train_loss': 0.08397273301045138, 'epoch': 1.0})

In [62]:
local_model_path = '/opt/app-root/src/aicoe-osc-demo/models/transformers/KPI_EXTRACTION'
trainer.save_model(local_model_path)

Saving model checkpoint to /opt/app-root/src/aicoe-osc-demo/models/transformers/KPI_EXTRACTION
Configuration saved in /opt/app-root/src/aicoe-osc-demo/models/transformers/KPI_EXTRACTION/config.json
Model weights saved in /opt/app-root/src/aicoe-osc-demo/models/transformers/KPI_EXTRACTION/pytorch_model.bin
tokenizer config file saved in /opt/app-root/src/aicoe-osc-demo/models/transformers/KPI_EXTRACTION/tokenizer_config.json
Special tokens file saved in /opt/app-root/src/aicoe-osc-demo/models/transformers/KPI_EXTRACTION/special_tokens_map.json


In [38]:
local_model_path = '/opt/app-root/src/aicoe-osc-demo/models/transformers/KPI_EXTRACTION'

tokenizer = AutoTokenizer.from_pretrained(local_model_path, use_fast=True)
model = AutoModelForQuestionAnswering.from_pretrained(local_model_path).to(device)

loading file vocab.txt
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading configuration file /opt/app-root/src/aicoe-osc-demo/models/transformers/KPI_EXTRACTION/config.json
Model config DistilBertConfig {
  "_name_or_path": "/opt/app-root/src/aicoe-osc-demo/models/transformers/KPI_EXTRACTION",
  "activation": "gelu",
  "architectures": [
    "DistilBertForQuestionAnswering"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.24.0",
  "vocab_size": 30522
}

loading weights file /opt/app-root/src/aicoe-osc-demo/models/transformers/KPI_EXTRACTION/pytorch_model.

## Evaluate

In [33]:
tokenized_datasets['test']

Dataset({
    features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 16928
})

In [39]:
for batch in trainer.get_test_dataloader(tokenized_datasets['test'].remove_columns(['start_positions', 'end_positions'])):
    break
batch = {k: v.to(trainer.args.device) for k, v in batch.items()}
with torch.no_grad():
    output = model(**batch)
output.keys()

odict_keys(['start_logits', 'end_logits'])

In [71]:
n_best_size = 20
start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1: -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1: -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        if start_index <= end_index:  # We need to refine that test to check the answer is inside the context
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": ""#  We need to find a way to get back the original
                            # substring corresponding to the answer in the context
                }
            )

In [44]:
def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [49]:
validation_features = climate_dataset["test"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=climate_dataset["test"].column_names
)

  0%|          | 0/17 [00:00<?, ?ba/s]

In [51]:
raw_predictions = trainer.predict(validation_features)

The following columns in the test set don't have a corresponding argument in `DistilBertForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping. If example_id, offset_mapping are not expected by `DistilBertForQuestionAnswering.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 16928
  Batch size = 16


In [52]:
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

In [56]:
max_answer_length = 30
start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
offset_mapping = validation_features[0]["offset_mapping"]
# The first feature comes from the first example. For the more general case, we will need to be match the example_id to
# an example index
context = climate_dataset["test"][0]["context"]

# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1: -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1: -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
        # to part of the input_ids that are not in the context.
        if (
            start_index >= len(offset_mapping)
            or end_index >= len(offset_mapping)
            or offset_mapping[start_index] is None
            or offset_mapping[end_index] is None
        ):
            continue
        # Don't consider answers with a length that is either < 0 or > max_answer_length.
        if end_index < start_index or end_index - start_index + 1 > max_answer_length:
            continue
        if start_index <= end_index: #  We need to refine that test to check the answer is inside the context
            start_char = offset_mapping[start_index][0]
            end_char = offset_mapping[end_index][1]
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": context[start_char: end_char]
                }
            )

valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:n_best_size]
valid_answers

[{'score': -7.7544737, 'text': 'paris agreement'},
 {'score': -8.830125,
  'text': 'paris agreement on climate change 2 degree goal and the political objectives arising from it, such as the eu’s 2030'},
 {'score': -8.8374405, 'text': 'paris agreement on climate change 2 degree'},
 {'score': -9.240417, 'text': 'paris'},
 {'score': -10.032496,
  'text': 'paris agreement on climate change 2 degree goal and the political objectives arising from it, such as the eu’s 2030 targets and #mission2030'},
 {'score': -10.074548,
  'text': 'paris agreement on climate change 2 degree goal and the political objectives arising from it, such as the eu'},
 {'score': -10.357061, 'text': 'agreement'},
 {'score': -10.363808, 'text': 'paris agreement on climate change'},
 {'score': -10.455936,
  'text': 'paris agreement on climate change 2 degree goal'},
 {'score': -10.580979,
  'text': '2 degree goal and the political objectives arising from it, such as the eu’s 2030'},
 {'score': -10.588295, 'text': '2 deg

In [58]:
examples = climate_dataset["test"]
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

In [61]:
def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []

        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )

        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}

        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
        predictions[example["id"]] = answer

    return predictions

In [62]:
final_predictions = postprocess_qa_predictions(climate_dataset["test"],
                                               validation_features,
                                               raw_predictions.predictions)

Post-processing 16891 example predictions split into 16928 features.


  0%|          | 0/16891 [00:00<?, ?it/s]

In [65]:
metric = load_metric("squad_v2")

/tmp/ipykernel_427/3306917064.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("squad_v2")


In [67]:
formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]

references = [{"id": ex["id"], "answers": ex["answers"]} for ex in climate_dataset["test"]]
metric.compute(predictions=formatted_predictions, references=references)

{'exact': 24.368006630750102,
 'f1': 24.473411087710208,
 'total': 16891,
 'HasAns_exact': 1.0714285714285714,
 'HasAns_f1': 7.429952437547432,
 'HasAns_total': 280,
 'NoAns_exact': 24.76070074047318,
 'NoAns_f1': 24.76070074047318,
 'NoAns_total': 16611,
 'best_exact': 98.34231247409863,
 'best_exact_thresh': 0.0,
 'best_f1': 98.34231247409863,
 'best_f1_thresh': 0.0}